In [2]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import json

products = pd.read_csv('../data/beers.csv', dtype={'Numero': object} )
beers = {}
for p in products['Numero']:
    with open(f'../data_raw/html/{p}.html', encoding="utf-8") as fp:
        try:
            soup = BeautifulSoup(fp, 'html.parser')
            try:
                taste_dec =[taste.text for taste in  soup.find_all("div", attrs={"class": "taste-description"})][0]
            except IndexError as e:
                taste_dec = str(e)
            foods = soup.find_all("div", re.compile("foodSymbol_"))
            try:
                food_tags = {food['class'][2].split('_',1)[1:][0] for food in foods}
            except IndexError as e:
                food_tags = {str(e)}
            #print(food_tags)
            #print(taste_dec)
            #print(soup.title.string)
            #print(soup.find_all("div", attrs={"class": "html-print"}))
            #print(soup.find_all("div", attrs={"class":"pdp main clearfix black js-accessibility-main-content"}))
            beerdata = {}
            beerdata['foods'] = list(food_tags)
            beerdata['taste_desc'] = taste_dec

            beers[p] = beerdata
        except UnicodeDecodeError:
            print("error")
            pass
        #print(taste_dec[0], type(taste_dec[0]), sep='\n')
        #print(str(foods))
with open(f'../data/beer_foods_tastes.json', 'w') as fo:
    json.dump(beers, fo,  indent=4)